In [ ]:
#creating cross validation and doing hyperparameter tuning

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os
df=pd.read_csv("/home/cdsw/train_for_mod2_v1.csv")
df_check=df
df_check["AV3Months"]=df.loc[: , "Shop_min3":"Shop_min1"].mean(axis=1)
df_check2=df_check[df_check.AV3Months >1]
df_check2.drop(df_check2.columns.to_series().loc["Unnamed: 0":"shop_id"],axis=1,inplace=True)
#split into train and test set
Y1=df_check2[["Target"]]
X1=df_check2.drop(["Target"],axis=1)

X_train1, X_test1,Y_train1, Y_test1 = train_test_split(X1,Y1,test_size=0.3, 
                                                   random_state=428)
with open("/home/cdsw/Models/xgb_1000", 'rb') as file:
    model = pickle.load(file)

[19:25:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [2]:
model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eta=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=8, min_child_weight=300, missing=nan, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             silent=None, subsample=0.8, verbosity=1)

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance
import time
ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train1, 
    Y_train1, 
    eval_metric="rmse", 
    eval_set=[(X_train1, Y_train1), (X_test1, Y_test1)], 
    verbose=True, 
    early_stopping_rounds = 10)

time.time() - ts

In [3]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train1, label=Y_train1)
dtest = xgb.DMatrix(X_test1, label=Y_test1)

In [9]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':0.3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    # Other parameters
    'objective':'reg:squarederror',
}
params['eval_metric'] = "rmse"
num_boost_round = 999


In [12]:
import time
ts = time.time()
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
time.time() - ts

[0]	Test-rmse:0.758955
Will train until Test-rmse hasn't improved in 10 rounds.
[1]	Test-rmse:0.684165
[2]	Test-rmse:0.638467
[3]	Test-rmse:0.611549
[4]	Test-rmse:0.59845
[5]	Test-rmse:0.591078
[6]	Test-rmse:0.582264
[7]	Test-rmse:0.581663
[8]	Test-rmse:0.578331
[9]	Test-rmse:0.578602
[10]	Test-rmse:0.577251
[11]	Test-rmse:0.579129
[12]	Test-rmse:0.580294
[13]	Test-rmse:0.579298
[14]	Test-rmse:0.579399
[15]	Test-rmse:0.579116
[16]	Test-rmse:0.579094
[17]	Test-rmse:0.579255
[18]	Test-rmse:0.579797
[19]	Test-rmse:0.578853
[20]	Test-rmse:0.578804
Stopping. Best iteration:
[10]	Test-rmse:0.577251



0.39528775215148926

In [13]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'rmse'},
    early_stopping_rounds=10
)
cv_results

,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,0.787885,0.087075,0.771570,0.020503
1,0.729321,0.086450,0.688592,0.018966
2,0.692727,0.085135,0.630650,0.017786
3,0.673419,0.085852,0.589800,0.017587
4,0.662138,0.084099,0.561189,0.014830
5,0.651879,0.078593,0.538476,0.012832
6,0.645035,0.076404,0.519994,0.014104
7,0.640938,0.075423,0.507223,0.013835
8,0.637529,0.071666,0.497037,0.013597
9,0.636585,0.070792,0.488364,0.013463


In [18]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(12,15)
]
gridsearch_params

[(9, 12),
 (9, 13),
 (9, 14),
 (10, 12),
 (10, 13),
 (10, 14),
 (11, 12),
 (11, 13),
 (11, 14)]

In [20]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in [6,9,12]
    for min_child_weight in [5,10,15]
]
gridsearch_params

[(6, 5),
 (6, 10),
 (6, 15),
 (9, 5),
 (9, 10),
 (9, 15),
 (12, 5),
 (12, 10),
 (12, 15)]

In [21]:
ts=time.time()
# Define initial best params and RMSE
min_rmse = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best RMSE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))
print("Time taken:", time.time()-ts)

CV with max_depth=6, min_child_weight=5
	RMSE 0.6057608 for 59 rounds
CV with max_depth=6, min_child_weight=10
	RMSE 0.604277 for 59 rounds
CV with max_depth=6, min_child_weight=15
	RMSE 0.6111996 for 41 rounds
CV with max_depth=9, min_child_weight=5
	RMSE 0.6127236 for 21 rounds
CV with max_depth=9, min_child_weight=10
	RMSE 0.607488 for 54 rounds
CV with max_depth=9, min_child_weight=15
	RMSE 0.6108084 for 31 rounds
CV with max_depth=12, min_child_weight=5
	RMSE 0.6172726000000001 for 12 rounds
CV with max_depth=12, min_child_weight=10
	RMSE 0.6075017999999999 for 27 rounds
CV with max_depth=12, min_child_weight=15
	RMSE 0.6134935999999999 for 22 rounds
Best params: 6, 10, RMSE: 0.604277
Time taken: 39.090760946273804


In [22]:
params['max_depth'] = 10
params['min_child_weight'] = 6
params

{'max_depth': 10,
 'min_child_weight': 6,
 'eta': 0.3,
 'subsample': 0.8,
 'colsample_bytree': 0.8,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse'}

In [23]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

In [24]:
ts=time.time()
# Define initial best params and RMSE
min_rmse = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )

    # Update best RMSE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = (subsample,colsample)
print("Best params: {}, {}, RMSE: {}".format(best_params[0], best_params[1], min_rmse))
print("Time taken:", time.time()-ts)

CV with subsample=1.0, colsample=1.0
	RMSE 0.6134198 for 13 rounds
CV with subsample=1.0, colsample=0.9
	RMSE 0.6181563999999999 for 15 rounds
CV with subsample=1.0, colsample=0.8
	RMSE 0.6094818 for 29 rounds
CV with subsample=1.0, colsample=0.7
	RMSE 0.604537 for 24 rounds
CV with subsample=0.9, colsample=1.0
	RMSE 0.6142122000000001 for 32 rounds
CV with subsample=0.9, colsample=0.9
	RMSE 0.6169414 for 11 rounds
CV with subsample=0.9, colsample=0.8
	RMSE 0.6115554 for 27 rounds
CV with subsample=0.9, colsample=0.7
	RMSE 0.6070708 for 31 rounds
CV with subsample=0.8, colsample=1.0
	RMSE 0.6187693999999999 for 9 rounds
CV with subsample=0.8, colsample=0.9
	RMSE 0.611535 for 18 rounds
CV with subsample=0.8, colsample=0.8
	RMSE 0.6106572 for 25 rounds
CV with subsample=0.8, colsample=0.7
	RMSE 0.6103778 for 27 rounds
CV with subsample=0.7, colsample=1.0
	RMSE 0.614842 for 9 rounds
CV with subsample=0.7, colsample=0.9
	RMSE 0.6135574 for 14 rounds
CV with subsample=0.7, colsample=0.8
	RM

In [25]:
params['subsample'] = 1
params['colsample_bytree'] = 0.7

In [30]:

# Define initial best params and RMSE
min_rmse = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best RMSE
    mean_rmse = cv_results['test-rmse-mean'].min()
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tRMSE {} for {} rounds".format(mean_rmse, boost_rounds))
    if mean_rmse < min_rmse:
        min_rmse = mean_rmse
        best_params = eta
print("Best params: {}, {}, RMSE: {}".format(best_params[0], min_rmse))

CV with eta=0.3
	RMSE 0.604537 for 24 rounds
CV with eta=0.2
	RMSE 0.6055889999999999 for 45 rounds
CV with eta=0.1
	RMSE 0.5961027999999999 for 45 rounds
CV with eta=0.05
	RMSE 0.5957838 for 90 rounds
CV with eta=0.01
	RMSE 0.594204 for 380 rounds
CV with eta=0.005
	RMSE 0.5929170000000001 for 846 rounds


TypeError: 'float' object is not subscriptable

In [31]:
#not the optimal value but will be quicker
params['eta'] = 0.05

In [32]:
import time
ts = time.time()
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
time.time() - ts

[0]	Test-rmse:0.847798
Will train until Test-rmse hasn't improved in 10 rounds.
[1]	Test-rmse:0.827184
[2]	Test-rmse:0.807814
[3]	Test-rmse:0.790019
[4]	Test-rmse:0.772685
[5]	Test-rmse:0.757243
[6]	Test-rmse:0.741536
[7]	Test-rmse:0.7281
[8]	Test-rmse:0.716152
[9]	Test-rmse:0.704233
[10]	Test-rmse:0.693598
[11]	Test-rmse:0.683108
[12]	Test-rmse:0.674831
[13]	Test-rmse:0.66685
[14]	Test-rmse:0.659193
[15]	Test-rmse:0.651765
[16]	Test-rmse:0.645795
[17]	Test-rmse:0.64007
[18]	Test-rmse:0.633489
[19]	Test-rmse:0.628215
[20]	Test-rmse:0.623394
[21]	Test-rmse:0.61922
[22]	Test-rmse:0.615688
[23]	Test-rmse:0.611901
[24]	Test-rmse:0.608377
[25]	Test-rmse:0.604634
[26]	Test-rmse:0.600346
[27]	Test-rmse:0.597667
[28]	Test-rmse:0.595165
[29]	Test-rmse:0.592604
[30]	Test-rmse:0.590687
[31]	Test-rmse:0.589499
[32]	Test-rmse:0.586966
[33]	Test-rmse:0.584981
[34]	Test-rmse:0.583535
[35]	Test-rmse:0.583054
[36]	Test-rmse:0.581869
[37]	Test-rmse:0.581008
[38]	Test-rmse:0.57886
[39]	Test-rmse:0.57792


2.4453580379486084

In [33]:
with open('/home/cdsw/Models/xgb_tuned', 'wb') as f:
    pickle.dump(model, f)

In [35]:
NovData2=pd.read_csv("/home/cdsw/Novdata_for_mod2_v2.csv")
NovData2_2=NovData2.drop(["Target"],axis="columns")
NovData2_2["AV3Months"]=NovData2_2.loc[: , "Shop_min3":"Shop_min1"].mean(axis=1)
NovData2_2.drop(NovData2_2.columns.to_series().loc["Unnamed: 0":"shop_id"],axis=1,inplace=True)
ylab=NovData2.Target
dval = xgb.DMatrix(NovData2_2, label=ylab)
#predict the results from the remaining data
preds2=pd.Series(model.predict(dval))

In [39]:
submission=pd.read_csv("/home/cdsw/01_future-sales/data/test.csv")
#Need to append to orig data sets
prep2=NovData2.loc[:,["shop_id","item_id"]]
prep2.reset_index(drop=True,inplace=True)
prep2["Sales_preds2"]=preds2

sub2=pd.merge(submission, prep2, how="left", on=["shop_id","item_id"])
sub3=sub2.fillna(0)
sub3.drop(["shop_id","item_id"],axis=1, inplace=True)
sub3.columns=["ID","item_cnt_month"]
sub3.to_csv("/home/cdsw/Submission_on_data2_xgb_tuned.csv",index=False)
sub3.head()

,ID,item_cnt_month
0,0,1.286050
1,1,0.000000
2,2,1.522582
3,3,1.420774
4,4,0.000000


In [41]:
orig=pd.read_csv("/home/cdsw/Submission_on_data2_xgb1000.csv")
orig.head()

,ID,item_cnt_month
0,0,1.401988
1,1,0.000000
2,2,1.744297
3,3,1.101410
4,4,0.000000
